# Alpha–Phi Binary Explorer (v5)

**What’s new (compared to v4)**  
* **Log‑scale depth map** `F_log` with divisor `e**(c·sqrt(n))` (smooth, no power‑of‑two jumps).  
* **Adaptive‑bias iterate** – small feedback term `eta = γ·(x/ALPHA−1)` inside the divisor.  
* **Tuned hybrid ping‑pong** with a `phi_scale` knob.  
* **Residence‑time histogram** to visualise shadow‑attractor bands.  
* **Jacobian & Lyapunov estimate** for diagnosing stability.  
* **Expanded parameter scan** sweeps (`p, q, lam, gamma, phi_scale`) and logs *only* hits < 1 %.  
Feel free to raise `mp.mp.dps` if you chase ppm accuracy.

In [9]:
import mpmath as mp, csv, matplotlib.pyplot as plt, collections, math
mp.mp.dps = 250

ALPHA = mp.mpf('7.2973525693e-3')
PHI   = (1 + mp.sqrt(5)) / 2

def fib_word_bits(n):
    a, b = '0', '01'
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

def fib_word_real(n):
    bits = fib_word_bits(n)
    return mp.fsum(int(b) * mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


## Log‑scale Depth Map

In [10]:
# --- Log‑scale depth map ---
def F_log(x, n_bits, p=1.6, c=0.7):
    return 1 / (x**p * mp.e**(c * mp.sqrt(n_bits)))

for c in (0.5, 0.7, 0.9):
    print(f'c={c}')
    for n in (64,128,256,512,1024,2048):
        y   = F_log(fib_word_real(n), n, p=1.6, c=c)
        err = abs((y-ALPHA)/ALPHA)
        print(f'  n={n:<4} rel_err={mp.nstr(err,3)}')
    print()


c=0.5
  n=64   rel_err=17.2
  n=128  rel_err=2.47
  n=256  rel_err=0.667
  n=512  rel_err=0.988
  n=1024 rel_err=1.0
  n=2048 rel_err=1.0

c=0.7
  n=64   rel_err=2.67
  n=128  rel_err=0.639
  n=256  rel_err=0.986
  n=512  rel_err=1.0
  n=1024 rel_err=1.0
  n=2048 rel_err=1.0

c=0.9
  n=64   rel_err=0.259
  n=128  rel_err=0.962
  n=256  rel_err=0.999
  n=512  rel_err=1.0
  n=1024 rel_err=1.0
  n=2048 rel_err=1.0



## Adaptive‑bias Iterate

In [11]:
# --- Adaptive‑bias iterate ---
def iterate_adapt(x0, steps=20, p=1.2, q=1.5, lam=0.08, gamma=0.02):
    traj=[x0]
    for _ in range(steps):
        eta  = gamma * (x0/ALPHA - 1)    # feedback term
        core = 1 / ((10**(q+eta)) * x0**p)
        x0   = core**lam
        traj.append(x0)
    return traj

seed = fib_word_real(512) * mp.sqrt(2)   # symmetry‑broken seed
traj = iterate_adapt(seed, steps=25)
for k,val in enumerate(traj):
    if k%5==0:
        print(f'k={k:<2} rel_err={mp.nstr(abs((val-ALPHA)/ALPHA),3)}')


k=0  rel_err=55.2
k=5  rel_err=80.6
k=10 rel_err=80.3
k=15 rel_err=80.3
k=20 rel_err=80.3
k=25 rel_err=80.3


## Tuned Hybrid Ping‑Pong

In [12]:
# --- Tuned hybrid ping‑pong ---
def hybrid_tuned(x0, cycles=12, q=2.0, comp_p=1.5, phi_scale=0.85):
    traj=[x0]
    for _ in range(cycles):
        x0 = (phi_scale*PHI) / x0
        x0 = 1 / (10**q * x0**comp_p)
        traj.append(x0)
    return traj

seed = fib_word_real(256)
traj = hybrid_tuned(seed)
print('cycle  rel_err')
for k,val in enumerate(traj):
    print(k, mp.nstr(abs((val-ALPHA)/ALPHA),3))


cycle  rel_err
0 38.8
1 0.867
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0


## Residence‑Time Histogram

In [13]:
# --- Residence‑time histogram ---
def band_index(x):
    if x < 1: return 0
    return int(math.floor((x-1)*10))     # bands: 1.0‑1.1 →0, 1.1‑1.2 →1 …

traj = iterate_adapt(fib_word_real(256), steps=200)
bands = [band_index(v/ALPHA) for v in traj]
hist = collections.Counter(bands)
print('Band index -> residence count (first 10 bands)')
for idx in range(10):
    print(idx, hist.get(idx,0))


Band index -> residence count (first 10 bands)
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0


## Jacobian & Finite‑time Lyapunov

In [14]:
# --- Jacobian & finite‑time Lyapunov at last 10 steps ---
def core(x, p, q, gamma):
    eta = gamma*(x/ALPHA-1)
    return 1 / ((10**(q+eta))*x**p)

def jacobian(x, p, q, lam, gamma):
    c = core(x,p,q,gamma)
    return lam * c**(lam-1) * (-p/x - math.log(10)*(q+gamma*(x/ALPHA-1))*c)

x_series = iterate_adapt(fib_word_real(512), steps=25, p=1.2, q=1.5, lam=0.08, gamma=0.02)
Js = [jacobian(x_series[i],1.2,1.5,0.08,0.02) for i in range(-10, -1)]
lyap = sum(math.log(abs(J)) for J in Js)/len(Js)
print('Mean |J| last 10 steps:', sum(abs(J) for J in Js)/9)
print('Finite‑time Lyapunov ≈', lyap)


Mean |J| last 10 steps: 65.84682297691034220445674092240260474601076432701891837535251480744639733971121096105037928061937044558191009962245811386931118558341260070923873808055839719746313378260594418719197222631422709829394121545746764626856838708904887590200997845060717614
Finite‑time Lyapunov ≈ 4.187331180661054


## Expanded Parameter Scan (combined iterate + hybrid)

In [15]:
# --- Wide parameter scan, hits <1% ---
hits=[]
with open('scan_log_v5.csv','w',newline='') as f:
    w=csv.writer(f); w.writerow(['p','q','lam','gamma','phi_scale','bits','rel_err'])
    for bits in (256,512,1024):
        seed=fib_word_real(bits)
        for p in [0.9,1.1,1.3,1.5]:
            for q in [1.2,1.5,2.0]:
                for lam in [0.05,0.08,0.12]:
                    for gamma in [0.01,0.02]:
                        for phi_scale in [0.8,0.85,0.9]:
                            x=iterate_adapt(seed,20,p,q,lam,gamma)[-1]
                            x=hybrid_tuned(x,cycles=6,q=q,comp_p=p,phi_scale=phi_scale)[-1] # Access the last element
                            rel=abs((x-ALPHA)/ALPHA)
                            if rel<0.01:
                                hits.append(rel)
                                w.writerow([p,q,lam,gamma,phi_scale,bits, mp.nstr(rel,12)])
print('Hits (<1%):', len(hits))

Hits (<1%): 0
